
# Django - Customise Your SQL



## Josh Smeaton
<hr>
- jarshwah on #django / Twitter / Github / Slack
- Software Engineer at Engage/LivePerson
- SQL Nerd
- Django Core Developer
- Primary author of Expressions


## Recent Developments
<hr>



## Lookups (1.7)
<hr />

```python
Product.objects.filter(
    tagarray__contains=['large', 'jumper']
)
```

## Transforms (1.7)
<hr />

```python
Product.objects.filter(
    name__lower__contains='jumper'
)
```

## Expressions (1.8)
<hr />

```python
Sale.objects.annotate(
    sale_month=TruncMonth('sale_date')
).values('sale_month').annotate(
    sales_per_month=Count('id')
)
```

# Sales Reporting
<hr />


In [7]:
from IPython.display import display
from shop.functions import table, sql

# Show sales and revenue per category
qs = Sale.objects.values('product__category').annotate(
    sales=Count('id'),
    revenue=Sum('sale_price')
)

display(table(qs))


,product__category,revenue,sales
0,Women,39359.00,332
1,Men,41281.00,340
2,Kids,10245.00,328


## We're not interested in Kids products
<hr />

![officespace](assets/office-space-1.jpg)

In [2]:
from IPython.display import display
from shop.functions import table, sql

# Show sales and revenue per category, but not Kids
qs = Sale.objects.exclude(
    product__category='Kids'
).values('product__category').annotate(
    sales=Count('id'),
    revenue=Sum('sale_price')
)

display(table(qs))

,product__category,revenue,sales
0,Women,39359.00,332
1,Men,41281.00,340


## WEBSCALE!
<hr>

![webscale](assets/office-space-2.jpg)

```sql
SELECT "shop_product"."category",
       SUM("shop_sale"."sale_price") AS "revenue",
       COUNT("shop_sale"."id") AS "sales"
FROM "shop_sale"
INNER JOIN "shop_product" 
ON ("shop_sale"."product_id" = "shop_product"."id")
WHERE NOT ("shop_product"."category" = 'Kids')
GROUP BY "shop_product"."category"
```

- "`NOT (condition = value)" vs "condition != value"` **5 characters!**
- Django has to do a lot more work for (correct) negated conditions

In [4]:
from django.db.models import Lookup
from django.db.models.fields import Field

@Field.register_lookup
class NotEqual(Lookup):
    lookup_name = 'ne'

    def as_sql(self, compiler, connection):
        lhs, lhs_params = self.process_lhs(
            compiler, connection)
        rhs, rhs_params = self.process_rhs(
            compiler, connection)
        params = lhs_params + rhs_params
        return '%s != %s' % (lhs, rhs), params


    

In [12]:
%timeit -c list(Product.objects.exclude(category='Kids'))
%timeit -c list(Product.objects.filter(category__ne='Kids'))

1000 loops, best of 3: 456 µs per loop
1000 loops, best of 3: 488 µs per loop


# Actually, can we see revenue by Quarter?
<hr />

![quarter](assets/office-space-3.png)

In [10]:
from datetime import datetime
from django.db.models import Transform
from django.db.models.fields import DateField, DateTimeField


@DateField.register_lookup
@DateTimeField.register_lookup
class TruncQuarter(Transform):
    function = 'DATE_TRUNC'
    template = "%(function)s('quarter', %(expressions)s)"
    lookup_name = 'quarter'
    
    def __init__(
        self, expression, output_field=DateTimeField(), **extra):
        super().__init__(
            expression, output_field=output_field, **extra)

In [ ]:
def convert_value(self, value, expression, connection, context):
    if ( isinstance(value, datetime) and
         type(self.output_field) == DateField):
        # we asked for a date but got a datetime
        value = value.date()
    return value

def as_sqlite(self, compiler, connection):
    raise NotImplementedError(
        'sqlite does not support truncating to Quarter.')

In [13]:
def resolve_expression(
        self, query=None, allow_joins=True, reuse=None,
        summarize=False, for_save=False
    ):
    copy = super().resolve_expression(
        query, allow_joins, reuse, summarize, for_save)
    
    input_field = copy.lhs.output_field
    if not isinstance(input_field, (DateField, DateTimeField)):
        raise ValueError(
        'Input expression must be DateField or DateTimeField'
        )
        
    output_field = copy.output_field
    if not isinstance(output_field, (DateField, DateTimeField)):
        raise ValueError(
        'output_field must be DateField or DateTimeField'
        )
    return copy

# Was all that necessary?!
<hr />


In [14]:
from datetime import datetime
from django.db.models import DateField, DateTimeField
from django.db.models.functions.datetime import TruncBase
from django.utils import timezone

@DateField.register_lookup
@DateTimeField.register_lookup
class TruncQtr(TruncBase):
    kind = 'quarter'
    lookup_name = 'qtr'

Sale.objects.filter(
    product__category__ne='Kids',
    sale_date__qtr__gte=datetime(2016, 1, 1, tzinfo=timezone.utc)
).count()

506

In [18]:
from IPython.display import display
from shop.functions import table, TruncQtr

qs = Sale.objects.filter(
    product__category__ne='Kids'
).annotate(quarter=TruncQtr('sale_date')
).values('quarter').annotate(
    revenue=Sum('sale_price'), sales=Count('id')
).order_by('quarter')

display(table(qs))

,quarter,revenue,sales
0,2015-07-01 00:00:00+00:00,5383.00,48
1,2015-10-01 00:00:00+00:00,14046.00,118
2,2016-01-01 00:00:00+00:00,28337.00,231
3,2016-04-01 00:00:00+00:00,32874.00,275


# You're a bit of a wizz! 
<hr />

## Now break out VIC and NSW individually.

![wizz](assets/office-space-4.jpg)

In [10]:
Sum(
    Case(
        When(state='VIC', then='sale_price'), 
        default=None
    )
)

Sum(CASE WHEN <Q: (AND: ('state', 'VIC'))> THEN F(sale_price), ELSE Value(None))

# That looks like SUMIF
<hr />

## We can do better.

![sumif](assets/office-space-5.jpg)

In [11]:
class SumIf(Sum):
    """
    Executes the equivalent of
        `Sum(Case(When(condition, then=field), default=None))`
    """
    def __init__(self, field, condition=None, **lookups):
        if lookups and condition is None:
            condition = Q(**lookups)
        case = Case(When(condition, then=field), default=None)
        super().__init__(case)

# Putting it altogether
<hr />

In [17]:
from IPython.display import display
from shop.functions import SumIf, table, TruncQtr

qs = Sale.objects.filter(product__category__ne='Kids'
).annotate(quarter=TruncQtr('sale_date')
).values('quarter').annotate(
    revenue=Sum('sale_price'), sales=Count('id'),
    rev_vic=SumIf('sale_price', state='VIC'),
    rev_nsw=SumIf('sale_price', state='NSW')
).order_by('quarter')

display(table(qs))


,quarter,rev_nsw,rev_vic,revenue,sales
0,2015-07-01 00:00:00+00:00,1585.50,2261.00,5383.00,48
1,2015-10-01 00:00:00+00:00,5559.50,4101.00,14046.00,118
2,2016-01-01 00:00:00+00:00,8550.50,10993.50,28337.00,231
3,2016-04-01 00:00:00+00:00,9886.50,13785.00,32874.00,275


# Hate it! Throw it away.

## Give me Quarter Over Quarter growth instead.

## That should be easy.

![startagain](assets/office-space-6.jpg)

In [3]:
analyticsql = """
SELECT
    quarter, revenue,
    round((
        revenue / LAG(revenue) over (ORDER BY quarter) - 1
    ) * 100, 2) || '%' rgrowth
FROM (
    SELECT
        DATE_TRUNC('quarter', s.sale_date) quarter,
        SUM(s.sale_price)                  revenue
    FROM shop_sale    s
    JOIN shop_product p ON s.product_id = p.id
    GROUP BY DATE_TRUNC('quarter', s.sale_date)
    ORDER BY quarter    
) quarterly_revenue"""

In [1]:
from IPython.display import display
from django.db import connection
from shop.functions import analyticsql, table

cursor = connection.cursor()
cursor.execute(analyticsql)
columns = [column[0] for column in cursor.description]
results = []
for row in cursor:
    results.append(dict(zip(columns, row)))
display(table(results))

,quarter,revenue,rgrowth
0,2015-07-01 00:00:00+00:00,5715.00,None
1,2015-10-01 00:00:00+00:00,16029.00,180.47%
2,2016-01-01 00:00:00+00:00,32137.00,100.49%
3,2016-04-01 00:00:00+00:00,37004.00,15.14%


# Thank You

![finished](assets/office-space-7.jpg)